In [15]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', '{:.2f}'.format)
all_data = pd.read_csv('/Users/bryan/Projects/finance-app/global-trends/backend/src/data/real_data.csv')
del all_data['Series Code']
del all_data['Country Code']
all_data.drop([32,33,34,35,36], inplace=True)
all_data.drop(17, inplace=True)
all_data.replace('..', np.nan, inplace=True)
for col in all_data.columns:
    if col not in ['Country Name', 'Series Name']:
        all_data[col] = pd.to_numeric(all_data[col], errors='coerce')

numeric_cols = all_data.drop(['Country Name', 'Series Name'], axis=1)
numeric_cols.interpolate(method='linear', axis=1, inplace=True, limit_direction='forward')
numeric_cols.bfill(axis=1, inplace=True)
mask = all_data.isna()
all_data[mask] = numeric_cols[mask]
replacements = {
    'GNI, Atlas method (current US$)': 'GNI',
    'GDP (current US$)': 'GDP',
    'Unemployment, total (% of total labor force) (national estimate)': 'Unemployment',
    'Inflation, consumer prices (annual %)': 'Inflation'
}

yearReplacements = {
    '2003 [YR2003]': 2003,
    '2004 [YR2004]': 2004,
    '2005 [YR2005]': 2005,
    '2006 [YR2006]': 2006,
    '2007 [YR2007]': 2007,
    '2008 [YR2008]': 2008,
    '2009 [YR2009]': 2009,
    '2010 [YR2010]': 2010,
    '2011 [YR2011]': 2011,
    '2012 [YR2012]': 2012,
    '2013 [YR2013]': 2013,
    '2014 [YR2014]': 2014,
    '2015 [YR2015]': 2015,
    '2016 [YR2016]': 2016,
    '2017 [YR2017]': 2017,
    '2018 [YR2018]': 2018,
    '2019 [YR2019]': 2019,
    '2020 [YR2020]': 2020,
    '2021 [YR2021]': 2021,
    '2022 [YR2022]': 2022,
}
all_data['Series Name'] = all_data['Series Name'].replace(replacements)
all_data.rename(columns=yearReplacements, inplace=True)
eap = all_data[all_data['Country Name'] == 'East Asia & Pacific']
del eap['Country Name']

eca = all_data[all_data['Country Name'] == 'Europe & Central Asia']
del eca['Country Name']

lac = all_data[all_data['Country Name'] == 'Latin America & Caribbean']
del lac['Country Name']

mena  = all_data[all_data['Country Name'] == 'Middle East & North Africa']
del mena['Country Name']

ssa = all_data[all_data['Country Name'] == 'Sub-Saharan Africa']
del ssa['Country Name']

world = all_data[all_data['Country Name'] == 'World']
del world['Country Name']

na = all_data[all_data['Country Name'] == 'North America']
del na['Country Name']

sa = all_data[all_data['Country Name'] == 'South Asia']
del sa['Country Name']

dataDict = {
    'world': world,
    'na': na,
    'eca': eca,
    'lac': lac,
    'ssa': ssa,
    'sa': sa,
    'eap': eap,
    'mena': mena
}
json_data = {}
for key,value in dataDict.items():
    json_data[key] = value.to_dict(orient='records')

ssa

,Series Name,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
16,GNI,495596652001.77,617569243404.95,784687635786.53,904951956605.77,1024167134549.86,1147980708606.07,1216227646535.81,1311174588177.22,1434274802058.79,...,1724040249317.91,1827757412494.77,1773224841084.61,1655297540599.80,1612698865717.05,1674380549989.40,1777074923477.38,1746335708547.77,1868198519423.47,1994737910616.27
18,GDP,558141609069.78,693283108166.62,823425641880.83,973649819318.58,1128673253084.49,1279544309271.44,1231672697171.96,1465215490467.22,1653394890750.09,...,1820439942754.10,1901497484757.43,1701776666914.07,1582166358879.25,1714024580927.10,1780679619928.75,1829596580378.40,1715842671714.98,1931458241338.34,2060531397309.08
19,Inflation,5.68,4.14,6.43,6.54,6.79,10.38,5.01,4.03,5.37,...,4.89,4.40,3.55,5.43,5.20,4.09,2.80,3.84,4.26,9.46
